In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Heart Disease/heart.csv')

In [6]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [7]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [ ]:
sklearn.metrics.accuracy_score(y_test, y_hat)

In [ ]:
import pickle
with open('heart_disease_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)